### **Data processing**
Contains code for processing raw files from DepMap into working format. Last edit 25.04.22.

In [2]:
import pandas as pd

In [3]:
lines = pd.read_csv('../data/raw/depmap_22q1_cellline_info_130422.csv')
expression = pd.read_csv('../data/raw/depmap_22q1_expression_trs_130422.csv')
dependency = pd.read_csv('../data/raw/depmap_22q1_crispr_effect_130422.csv')
essential = pd.read_csv('../data/raw/depmap_22q1_crispr_essential_230422.csv')
annot = pd.read_csv('../data/raw/ensembl_trs_dict_240422.txt', sep='\t')

In [4]:
expression['Unnamed: 0'] = expression['Unnamed: 0'].map(lines.set_index('DepMap_ID')['stripped_cell_line_name'])
dependency['DepMap_ID'] = dependency['DepMap_ID'].map(lines.set_index('DepMap_ID')['stripped_cell_line_name'])

In [5]:
dependency = dependency[dependency['DepMap_ID'].isin(expression['Unnamed: 0'])]
expression = expression[expression['Unnamed: 0'].isin(dependency['DepMap_ID'])]

In [6]:
dependency = dependency.set_index('DepMap_ID', drop=True).T
dependency.columns.name = None
dependency.index = dependency.index.map(lambda x: x[:x.index('(')-1])
dependency.index.name = 'gene_name'

In [7]:
expression = expression.set_index('Unnamed: 0', drop=True).T
expression.columns.name = None
expression = expression[expression.index.str.contains('\(ENST')]
expression['gene_name'] = expression.index.map(lambda x: x[:x.index('(')-1])
expression['enst_id'] = expression.index.map(lambda x: x[x.index('(')+1:-1])
expression = expression.reset_index().drop('index', axis=1)
expression = expression[expression['gene_name'].isin(dependency.index)]
expression = expression.set_index(['gene_name', 'enst_id'], drop=True)

In [8]:
dependency = dependency[sorted(dependency.columns)]
expression = expression[sorted(expression.columns)]
expression = expression.sort_index()

In [9]:
annot = annot.drop_duplicates('Transcript stable ID', keep='first')
expression['trs_type'] = expression.index.get_level_values(1).map(annot.set_index('Transcript stable ID')['Transcript type'])

In [10]:
essential['gene'] = essential['gene'].map(lambda x: x[:x.index('(')-1]) #Get rid of Entrez ID

In [14]:
expression.to_csv('../data/processed/depmap_22q1_expression_trs_processed_240422.csv')
dependency.to_csv('../data/processed/depmap_22q1_crispr_effect_processed_240422.csv')
essential.to_csv('../data/processed/depmap_22q1_crispr_essential_processed_240422.csv', index=False)